<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Inverse-α-&amp;-Inverse-β" data-toc-modified-id="Inverse-α-&amp;-Inverse-β-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Inverse α &amp; Inverse β</a></span></li><li><span><a href="#Calculate-by-Statsmodels" data-toc-modified-id="Calculate-by-Statsmodels-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Calculate by Statsmodels</a></span><ul class="toc-item"><li><span><a href="#Beta" data-toc-modified-id="Beta-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Beta</a></span></li><li><span><a href="#Raw-Effect-Size" data-toc-modified-id="Raw-Effect-Size-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Raw Effect Size</a></span></li><li><span><a href="#Sample-Size" data-toc-modified-id="Sample-Size-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Sample Size</a></span></li></ul></li><li><span><a href="#Calculate-by-Simulation" data-toc-modified-id="Calculate-by-Simulation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Calculate by Simulation</a></span><ul class="toc-item"><li><span><a href="#Beta" data-toc-modified-id="Beta-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Beta</a></span></li><li><span><a href="#Raw-Effect-Size" data-toc-modified-id="Raw-Effect-Size-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Raw Effect Size</a></span></li><li><span><a href="#Sample-Size" data-toc-modified-id="Sample-Size-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Sample Size</a></span></li></ul></li></ul></div>

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import IPython as ip
mpl.style.use('ggplot')
mpl.rc('font', family='Noto Sans CJK TC')
ip.display.set_matplotlib_formats('svg')

# Setup

In [2]:
anr = 0.5  # actual negative rate
alpha = 0.05
beta = 0.2

# n: sample size
n_1 = 100
n_2 = 100

# sm: sample mean
# ss: sample standard deviation
sm_1 = 170
ss_1 = 5
sm_2 = 170+2
ss_2 = 5

# https://en.wikipedia.org/wiki/Effect_size#Cohen's_d
raw_effect_size = sm_2-sm_1
ss_pooled = np.sqrt(
    ((n_1-1)*ss_1**2 + (n_2-1)*ss_2**2) /
    (n_1+n_2-2)
)
cohen_effect_size = raw_effect_size / ss_pooled

# Inverse α & Inverse β

$
{\displaystyle \text{false discovery rate} = \frac{ \text{false positive rate} \cdot \text{actual negative rate} }{\text{predicted positive rate}}}
$

$
{\displaystyle \text{false omission rate} = \frac{ \text{false negative rate} \cdot \text{actual positive rate} }{\text{predicted negative rate}}}
$

$
{\displaystyle \text{inverse }\alpha = \frac{ \alpha \cdot \text{actual negative rate} }{\text{predicted positive rate}}}
$

$
{\displaystyle \text{inverse }\beta = \frac{ \beta \cdot \text{actual positive rate} }{\text{predicted negative rate}}}
$

In [3]:
# anr = 0.95
# alpha = 0.05
# beta = 0.05
# # inverse_alpha -> 0.5000
# # inverse_beta -> 0.0028

# anr = 0.995
# alpha = 0.01
# beta = 0.01
# # inverse_alpha -> 0.6678
# # inverse_beta -> 0.00005

apr = 1-anr  # actual positive rate
power = 1-beta
cl = 1-alpha  # confidence level

ppr = alpha*anr + power*apr  # predicted positive rate = P(+) = P(+|H_0)*P(H_0)+P(+|H_1)*P(H_1)
pnr = cl*anr + beta*apr  # predicted positive rate = P(-)

inverse_alpha = alpha*anr / ppr
inverse_beta = beta*apr / pnr

print(inverse_alpha)
print(inverse_beta)

0.058823529411764705
0.1739130434782609


# Calculate by Statsmodels

## Beta

In [4]:
%%time
1-sm.stats.tt_ind_solve_power(
    alpha=alpha,
    effect_size=cohen_effect_size,
    nobs1=n_1,
    ratio=n_2/n_1,
    power=None,
)

CPU times: user 599 µs, sys: 131 µs, total: 730 µs
Wall time: 644 µs


0.19635250345692312

## Raw Effect Size

In [5]:
%%time
sm.stats.tt_ind_solve_power(
    alpha=alpha,
    power=1-beta,
    effect_size=None,
    nobs1=n_1,
    ratio=n_2/n_1,
)*ss_pooled

CPU times: user 5.78 ms, sys: 1.21 ms, total: 6.99 ms
Wall time: 5.96 ms


1.9906955869556378

## Sample Size

In [6]:
%%time
sm.stats.tt_ind_solve_power(
    alpha=alpha,
    power=1-beta,
    effect_size=cohen_effect_size,
    ratio=1, # = n_2 / n_1
    nobs1=None,
)

CPU times: user 7.54 ms, sys: 1.53 ms, total: 9.07 ms
Wall time: 7.77 ms


99.08032683981143

# Calculate by Simulation

In [7]:
n_sim = 1000

## Beta

In [8]:
%%time
np.random.seed(20180702)
sample_1_m = sp.stats.norm.rvs(loc=sm_1, scale=ss_1, size=(n_1, n_sim))
sample_2_m = sp.stats.norm.rvs(loc=sm_2, scale=ss_2, size=(n_2, n_sim))
beta = (sp.stats.ttest_ind(sample_1_m, sample_2_m).pvalue >= alpha).sum() / n_sim
print(beta)

0.214
CPU times: user 11.1 ms, sys: 3.49 ms, total: 14.6 ms
Wall time: 17.6 ms


## Raw Effect Size

In [9]:
def calc_beta_given_raw_effect_size(x):
    np.random.seed(20180702)
    sample_1_m = sp.stats.norm.rvs(loc=sm_1, scale=ss_1, size=(n_1, n_sim))
    # assume the sample 2 has the sample standard deviation
    sample_2_m = sp.stats.norm.rvs(loc=sm_1+x, scale=ss_1, size=(n_2, n_sim))
    beta = (sp.stats.ttest_ind(sample_1_m, sample_2_m).pvalue >= alpha).sum() / n_sim
    return beta

In [10]:
%%time
# === given required beta, find the raw effect size between 3 and 0
sp.optimize.brentq(
    lambda x: calc_beta_given_raw_effect_size(x)-beta,
    3, 0
)

CPU times: user 50.5 ms, sys: 1.98 ms, total: 52.4 ms
Wall time: 51.3 ms


1.995603474483318

## Sample Size

In [11]:
def calc_beta_given_sample_size(x):
    np.random.seed(20180702)
    sample_1_m = sp.stats.norm.rvs(loc=sm_1, scale=ss_1, size=(int(x), n_sim))
    sample_2_m = sp.stats.norm.rvs(loc=sm_2, scale=ss_2, size=(int(x), n_sim))
    beta = (sp.stats.ttest_ind(sample_1_m, sample_2_m).pvalue >= alpha).sum() / n_sim
    return beta

In [12]:
%%time
# === given required beta, find the sample size between 120, 80
sp.optimize.brentq(
    lambda x: calc_beta_given_sample_size(x)-beta,
    120, 80
)

CPU times: user 29.5 ms, sys: 2.16 ms, total: 31.6 ms
Wall time: 30.2 ms


100.89011695437664